In [ ]:
from pv_analyzer import pv_analyze
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
%matplotlib widget

In [ ]:
def emp_kep_v(v_rot, v_100):        
        r_au = 100.*np.power(v_100/v_rot, 2)
        return r_au

def emp_kep_r(r, r_b, v_100):
        v_r = v_100*(r/r_b)**(-0.5)
        return v_r

In [ ]:
# define fix r points
# Get equidistance points

r_au_empkep_rs_fix_r = np.arange(start=42., step=42., stop=840.)
r_au_empkep_bs_fix_r = -r_au_empkep_rs_fix_r

v_100 = 2.842 #kmps # idpft-2.674 # JS- 3.2 # newft 2.842

v_rot_empkep_rs_fix_r =  emp_kep_r(r=r_au_empkep_rs_fix_r, r_b=100., v_100=v_100)
v_rot_empkep_bs_fix_r = - v_rot_empkep_rs_fix_r

plot_points = [r_au_empkep_rs_fix_r, v_rot_empkep_rs_fix_r, r_au_empkep_bs_fix_r, v_rot_empkep_bs_fix_r]




In [ ]:
pv_without_beam_fname = "L1489irs_model_i_73.0without_beam_conv_upd_hdr_pv.fits"

pv_with_beam_fname = "L1489irs_model_i_73.0with_beam_conv_upd_hdr_pv.fits"

pv_with_beam = pv_analyze(pv_path=pv_with_beam_fname,
                    is_Tb=False, v_sys=7.22) # V_sys - 7.22, 7.27 7.33
pv_with_beam.read_pv(print_details = True)          
Tb_pv_with_beam = pv_with_beam.get_tb_on_curve(curve_function = None, 
                            get_surrounding_pix = False, mode='func',
                            num_pix = 3, return_coords=True)


pv_without_beam = pv_analyze(pv_path=pv_without_beam_fname,
                    is_Tb=False, v_sys=7.22) # V_sys - 7.22, 7.27 7.33
pv_without_beam.read_pv(print_details = True)


Tb_pv_without_beam = pv_without_beam.get_tb_on_curve(curve_function = None, 
                            get_surrounding_pix = False, mode='func',
                            num_pix = 3, return_coords=True)

In [ ]:
pv_without_beam_fig = pv_without_beam.plot_pv(plot_curve=True, base_ctr_lvl= 0.2,
                                              cbarlim=[0.,0.10])


In [ ]:
print(Tb_pv_without_beam["r_rs"])
print(Tb_pv_without_beam["Tb_on_point_rs"])

In [ ]:
pv_with_beam_fig = pv_with_beam.plot_pv(plot_curve=True, base_ctr_lvl= 0.1)

In [ ]:
print(Tb_pv_with_beam["Tb_on_point_rs"])

In [ ]:
fig, axs = plt.subplots()

def ssdisk(r, Ic, rc, gamma, beta = None):
    beta_p = gamma if beta is None else beta # - beta = - gamma - q
    
    I_cube = Ic * (r/rc)**(- beta_p) * np.exp(-(r/rc)**(2. - gamma))
    
    I_cube *= np.abs((0.06)*(0.06)) # per pixel to per arcsec^2
    I_cube /= np.pi/(4.*np.log(2.)) * 0.6 * 0.6 # per arcsec^2 --> per beam
    
    return I_cube

cf = (np.pi/(4.*np.log(2.)) * 0.6 * 0.6)/np.abs((0.03)*(0.03))

#Tb_pv_without_beam["Tb_on_point_rs"] = ssdisk(r=Tb_pv_without_beam["r_rs"],
#                                              Ic=1.,rc=600., beta=1.5, gamma=1.)
#Tb_pv_without_beam["Tb_on_point_bs"] = ssdisk(r=-Tb_pv_without_beam["r_bs"],
#                                              Ic=1.,rc=600., beta=1.5, gamma=1.)
#print(Tb_pv_without_beam["Tb_on_point_bs"])

Tb_pv_without_beam["Tb_on_point_rs"] = np.array(Tb_pv_without_beam["Tb_on_point_rs"])*cf
Tb_pv_without_beam["Tb_on_point_bs"] = np.array(Tb_pv_without_beam["Tb_on_point_bs"])*cf

axs.plot(Tb_pv_without_beam["r_rs"], Tb_pv_without_beam["Tb_on_point_rs"], 
         label='without_beam_rs', c='r', ls = '-', marker='o')
axs.plot(Tb_pv_with_beam["r_rs"], Tb_pv_with_beam["Tb_on_point_rs"], label='with_beam_rs', c='r', ls = '--', marker='o')

axs.plot(-Tb_pv_without_beam["r_bs"], Tb_pv_without_beam["Tb_on_point_bs"], 
         label='without_beam_bs', c='b', ls = '-', marker='o')
axs.plot(-Tb_pv_with_beam["r_bs"], Tb_pv_with_beam["Tb_on_point_bs"], 
         label='with_beam_bs', c='b', ls = '--', marker='o')
axs.legend(prop={'size': 6})
axs.grid()
axs.set_yscale('log')
plt.show()

In [ ]:
red_factor = np.divide(Tb_pv_without_beam["Tb_on_point_rs"],Tb_pv_with_beam["Tb_on_point_rs"])

blue_factor = np.divide(Tb_pv_without_beam["Tb_on_point_bs"],Tb_pv_with_beam["Tb_on_point_bs"])



fig, axs = plt.subplots()
axs.scatter(Tb_pv_without_beam["r_rs"], 1/red_factor, label='ratio_rs', c='r', ls = '-')
axs.scatter(-Tb_pv_with_beam["r_bs"], 1/blue_factor, label='ratio_bs', c='b', ls = '--')
axs.legend()
axs.set_yscale('log')
axs.grid()
plt.show()